# ML Course 2023 |  Sentiment Analysis in Twitter Challenge
You can check the updated leaderboard in this [link](https://nimble-hellebore-184.notion.site/ML-Course-2023-Sentiment-Analysis-in-Twitter-Challenge-966b041e7aec4f2eabbc8dc33d64b871).

In [1]:
!pip3 install tueplots==0.0.5
!pip3 install sentence-transformers==2.2.2
!pip3 install datasets
!pip3 install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=68c36825b8c3b6f01a49528a3b629ed8cb612f7fe6f8bec393ec771402682637
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta

In [2]:
import os
import pandas as pd

from sentence_transformers import SentenceTransformer

In [7]:
from google.colab import drive
drive.mount('/content/drive')
#CHANGE THIS PATH FOR YOU
%cd '/content/drive/MyDrive/ML Twitter Project/Project Tutorials'
# %cd '/content/drive/MyDrive/coreML23/ML_2023/students'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/14NNtlza32SBywYBzD2sUNSqh1WjSRSEm/ML Twitter Project/Project Tutorials


In [8]:
team_id = '36' #put your team id here
split = 'test_1' # replace by 'test_2' for FINAL submission

df = pd.read_csv('tweets_train.csv') # .sample(100) # we are subsampling 100 training data
df_test = pd.read_csv(f'tweets_{split}.csv') # .sample(200) # we are subsampling 200 test data
df_test_2 = pd.read_csv(f'tweets_test_2.csv') # .sample(200) # we are subsampling 200 test data


In [9]:
### PREPROCESSING IS ALREADY DONE AND WE GET WORDS COLUMN ###

# joining list of words to create sentences

df['words_str'] = df['words'].apply(lambda words: ' '.join(eval(words)))
df_test['words_str'] = df_test['words'].apply(lambda words: ' '.join(eval(words)))
df_test_2['words_str'] = df_test_2['words'].apply(lambda words: ' '.join(eval(words)))


In [10]:
from datasets import Dataset
import numpy as np

df_train, df_val, df_check_test = \
              np.split(df.sample(frac=1, random_state=42),
                       [int(.8*len(df)), int(.9*len(df))])

In [11]:
raw_train_ds = Dataset.from_pandas(df_train)
raw_val_ds = Dataset.from_pandas(df_val)
raw_test_ds = Dataset.from_pandas(df_check_test)

# This line prints the description of train_ds
raw_train_ds, raw_val_ds, raw_test_ds

(Dataset({
     features: ['id', 'text', 'type', 'author_id', 'possibly_sensitive', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'words', 'score_compound', 'sentiment', 'words_str', '__index_level_0__'],
     num_rows: 6400
 }),
 Dataset({
     features: ['id', 'text', 'type', 'author_id', 'possibly_sensitive', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'words', 'score_compound', 'sentiment', 'words_str', '__index_level_0__'],
     num_rows: 800
 }),
 Dataset({
     features: ['id', 'text', 'type', 'author_id', 'possibly_sensitive', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'words', 'score_compound', 'sentiment', 'words_str', '__index_level_0__'],
     num_rows: 800
 }))

In [12]:
LABEL_DICTIONARY = {
    "positive": "2",
    "neutral": "1",
    "negative": "0"
}

id2label = {k:l for k, l in enumerate(LABEL_DICTIONARY)}
label2id = {l:k for k, l in enumerate(LABEL_DICTIONARY)}

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader

BASE_MODEL = "bert-base-uncased"
LEARNING_RATE = 2e-5
MAX_LENGTH = 256
BATCH_SIZE = 16
EPOCHS = 5

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, id2label=id2label, label2id=label2id)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
ds = {"train": raw_train_ds, "validation": raw_val_ds, "test": raw_test_ds}

def preprocess_function(examples: dict):
    labels = [0] * len(id2label)
    for k, l in id2label.items():
        if l == examples["sentiment"]:
            labels[k] = 1
        else:
            labels[k] = 0

    examples = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    examples["label"] = labels

    return examples


In [15]:
for split in ds:
    ds[split] = ds[split].map(preprocess_function, remove_columns=['id', 'text', 'type', 'author_id', 'possibly_sensitive', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'words', 'score_compound', 'sentiment', 'words_str'])

ds

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

{'train': Dataset({
     features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
     num_rows: 6400
 }),
 'validation': Dataset({
     features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
     num_rows: 800
 }),
 'test': Dataset({
     features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
     num_rows: 800
 })}

In [71]:
GLOBAL_SCORE_LABELS = ["positive", "neutral", "negative"]
GLOBAL_SCORE_INDICES = range(0, 2)


from datasets import load_metric

accuracy = load_metric("accuracy")

def get_preds_from_logits_2(logits):
    ret = np.zeros(logits.shape)

    # The first 5 columns (GLOBAL_SCORE_INDICES) are for global scores. They should be handled with a multiclass approach
    # i.e. we fill 1 to the class with highest probability, and 0 into the other columns
    best_class = np.argmax(logits[:, GLOBAL_SCORE_INDICES], axis=-1)
    ret[list(range(len(ret))), best_class] = 1

    return best_class, ret


def get_preds_from_logits(logits):
    ret = np.zeros(logits.shape)

    # The first 5 columns (GLOBAL_SCORE_INDICES) are for global scores. They should be handled with a multiclass approach
    # i.e. we fill 1 to the class with highest probability, and 0 into the other columns
    best_class = np.argmax(logits[:, GLOBAL_SCORE_INDICES], axis=-1)
    ret[list(range(len(ret))), best_class] = 1

    return ret

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    final_metrics = {}

    # Deduce predictions from logits
    predictions = get_preds_from_logits(logits)

    # Get f1 metrics for global scoring. Notice that f1_micro = accuracy
    final_metrics["f1_micro_for_global_score"] = f1_score(labels[:, GLOBAL_SCORE_INDICES], predictions[:, GLOBAL_SCORE_INDICES], average="micro")
    final_metrics["f1_macro_for_global_score"] = f1_score(labels[:, GLOBAL_SCORE_INDICES], predictions[:, GLOBAL_SCORE_INDICES], average="macro")

    # The global f1_metrics
    final_metrics["f1_micro"] = f1_score(labels, predictions, average="micro")
    final_metrics["f1_macro"] = f1_score(labels, predictions, average="macro")

    final_metrics["accuracy"] = 0

    # Classification report
    print("Classification report for global scores: ")
    print(classification_report(labels[:, GLOBAL_SCORE_INDICES], predictions[:, GLOBAL_SCORE_INDICES], zero_division=0))
    return final_metrics

In [72]:
import torch
import pandas as pd
import numpy as np
import json
import torch

from datasets import Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainerCallback, TrainingArguments
from torch.utils.data import DataLoader

from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report

class MultiTaskClassificationTrainer(Trainer):
    def __init__(self, group_weights=None, **kwargs):
        super().__init__(**kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]

        global_score_loss = torch.nn.functional.cross_entropy(logits[:, GLOBAL_SCORE_INDICES], labels[:, GLOBAL_SCORE_INDICES])

        loss = global_score_loss
        return (loss, outputs) if return_outputs else loss

In [76]:
training_args = TrainingArguments(
    output_dir="./models/bert-base-sentiment-latest",
    learning_rate=2e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=30,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    weight_decay=0.01,
)

trainer = MultiTaskClassificationTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    compute_metrics=compute_metrics,
    model_init=model_init
)

/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:357: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(


In [77]:
# trainer.eval_dataset=ds["test"]
# trainer.evaluate()

In [78]:
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1 Micro For Global Score,F1 Macro For Global Score,F1 Micro,F1 Macro,Accuracy
1,No log,0.317957,0.831504,0.790591,0.805000,0.527061,0
2,No log,0.318315,0.840542,0.812380,0.813750,0.541586,0
3,0.295900,0.357034,0.845707,0.814067,0.818750,0.542711,0
4,0.295900,0.590241,0.826340,0.800120,0.800000,0.533413,0
5,0.080300,0.675343,0.830213,0.802790,0.803750,0.535193,0
6,0.080300,0.720343,0.837960,0.809613,0.811250,0.539742,0
7,0.080300,0.787688,0.836669,0.811008,0.810000,0.540672,0
8,0.017300,0.785815,0.846998,0.813849,0.820000,0.542566,0
9,0.017300,0.891036,0.841833,0.809724,0.815000,0.539816,0
10,0.008700,0.892520,0.845707,0.816042,0.818750,0.544028,0


Classification report for global scores: 
              precision    recall  f1-score   support

           0       0.77      0.66      0.71       204
           1       0.81      0.94      0.87       545

   micro avg       0.81      0.86      0.83       749
   macro avg       0.79      0.80      0.79       749
weighted avg       0.80      0.86      0.83       749
 samples avg       0.81      0.81      0.81       749

Classification report for global scores: 
              precision    recall  f1-score   support

           0       0.73      0.77      0.75       204
           1       0.84      0.91      0.87       545

   micro avg       0.81      0.87      0.84       749
   macro avg       0.79      0.84      0.81       749
weighted avg       0.81      0.87      0.84       749
 samples avg       0.81      0.81      0.81       749

Classification report for global scores: 
              precision    recall  f1-score   support

           0       0.78      0.72      0.75       204
   

TrainOutput(global_step=6000, training_loss=0.03519228826587399, metrics={'train_runtime': 8588.3617, 'train_samples_per_second': 22.356, 'train_steps_per_second': 0.699, 'total_flos': 2.5258888101888e+16, 'train_loss': 0.03519228826587399, 'epoch': 30.0})

In [79]:
trainer.eval_dataset=ds["test"]
trainer.evaluate()
trainer.save_model()
import pandas as pd
df_test_2 = pd.read_csv(f'tweets_test_2.csv') # .sample(200) # we are subsampling 200 test data
df_test_2['words_str'] = df_test_2['words'].apply(lambda words: ' '.join(eval(words)))

raw_test_final = Dataset.from_pandas(df_test_2)

ds_test = {"test": raw_test_final}
import math
import pandas as pd

y_preds = []

for i in range(len(raw_test_final)):
    input_texts = raw_test_final[i]["text"]
    encoded = tokenizer(input_texts, truncation=True, padding="max_length", max_length=256, return_tensors="pt").to("cuda")
    l = model(**encoded).logits.detach().to('cpu').numpy()
    c, p = get_preds_from_logits_2(l)
    y_preds.append(id2label[c.tolist()[0]])

print(y_preds)

Classification report for global scores: 
              precision    recall  f1-score   support

           0       0.81      0.69      0.74       223
           1       0.82      0.93      0.87       533

   micro avg       0.81      0.86      0.84       756
   macro avg       0.81      0.81      0.81       756
weighted avg       0.81      0.86      0.83       756
 samples avg       0.81      0.81      0.81       756

['positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'neu

In [80]:
y_preds

['positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'posi

In [86]:
import pandas as pd
df_test_2 = pd.read_csv(f'tweets_train.csv').sample(200) # we are subsampling 200 test data
df_test_2['words_str'] = df_test_2['words'].apply(lambda words: ' '.join(eval(words)))

raw_test_final = Dataset.from_pandas(df_test_2)

ds_test = {"test": raw_test_final}
import math
import pandas as pd

y_preds = []

for i in range(len(raw_test_final)):
    input_texts = raw_test_final[i]["text"]
    encoded = tokenizer(input_texts, truncation=True, padding="max_length", max_length=256, return_tensors="pt").to("cuda")
    l = model(**encoded).logits.detach().to('cpu').numpy()
    c, p = get_preds_from_logits_2(l)
    y_preds.append(id2label[c.tolist()[0]])

print(y_preds)

['neutral', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'positive

In [107]:
ds['test']

Dataset({
    features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 800
})

In [98]:
trainer.predict(ds['test'])

Classification report for global scores: 
              precision    recall  f1-score   support

           0       0.81      0.69      0.74       223
           1       0.82      0.93      0.87       533

   micro avg       0.81      0.86      0.84       756
   macro avg       0.81      0.81      0.81       756
weighted avg       0.81      0.86      0.83       756
 samples avg       0.81      0.81      0.81       756



PredictionOutput(predictions=array([[-1.6770142 ,  2.2442827 , -0.1708987 ],
       [-1.062876  ,  1.3853391 ,  0.25137442],
       [ 0.43953353, -0.64419013,  0.5948939 ],
       ...,
       [-1.6735641 ,  2.4975672 , -0.4362247 ],
       [-1.4584515 ,  2.172785  , -0.27444935],
       [ 0.96232915, -1.1624359 ,  0.51451695]], dtype=float32), label_ids=array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32), metrics={'test_loss': 0.31705474853515625, 'test_f1_micro_for_global_score': 0.8380462724935733, 'test_f1_macro_for_global_score': 0.8080577341218473, 'test_f1_micro': 0.815, 'test_f1_macro': 0.5387051560812316, 'test_accuracy': 0, 'test_runtime': 11.7018, 'test_samples_per_second': 68.366, 'test_steps_per_second': 2.136})

In [92]:
classification_report(raw_test_final['sentiment'], y_preds)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n    negative       0.00      0.00      0.00        12\n     neutral       0.63      0.65      0.64       127\n    positive       0.26      0.30      0.27        61\n\n    accuracy                           0.50       200\n   macro avg       0.30      0.31      0.30       200\nweighted avg       0.48      0.50      0.49       200\n'

In [118]:
import pandas as pd
df_test_2 = pd.read_csv(f'tweets_train.csv') #.sample(200) # we are subsampling 200 test data
df_test_2['words_str'] = df_test_2['words'].apply(lambda words: ' '.join(eval(words)))

def preprocess_function_2(examples: dict):
    labels = [0] * len(id2label)
    for k, l in id2label.items():
        if l == examples["sentiment"]:
            labels[k] = 1
        else:
            labels[k] = 0

    examples = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    examples["label"] = labels

    return examples


raw_test_final = Dataset.from_pandas(df_test_2)
raw_test_final = raw_test_final.map(preprocess_function_2, remove_columns=['id', 'text', 'type', 'author_id', 'possibly_sensitive', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'words', 'score_compound', 'sentiment', 'words_str'])
res = trainer.predict(raw_test_final)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Classification report for global scores: 
              precision    recall  f1-score   support

           0       0.85      0.75      0.79      2228
           1       0.83      0.95      0.89      5290

   micro avg       0.84      0.89      0.86      7518
   macro avg       0.84      0.85      0.84      7518
weighted avg       0.84      0.89      0.86      7518
 samples avg       0.84      0.84      0.84      7518



In [119]:
p = []
for i in res.predictions:
   p.append(np.argmax(i))

In [120]:
p

[1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 0,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 0,
 1,
 2,
 2,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 2,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,


In [121]:
r = []
for i in res.label_ids:
  r.append(np.argmax(i))

In [122]:
classification_report(r, p)

'              precision    recall  f1-score   support\n\n           0       0.94      0.58      0.71      2228\n           1       0.86      0.90      0.88      5290\n           2       0.05      0.11      0.07       482\n\n    accuracy                           0.76      8000\n   macro avg       0.61      0.53      0.55      8000\nweighted avg       0.83      0.76      0.78      8000\n'

In [ ]:
import pandas as pd
df_test_2 = pd.read_csv(f'tweets_test_1.csv') #.sample(200) # we are subsampling 200 test data
df_test_2['words_str'] = df_test_2['words'].apply(lambda words: ' '.join(eval(words)))

raw_test_final = Dataset.from_pandas(df_test_2)
raw_test_final = raw_test_final.map(preprocess_function, remove_columns=['id', 'text', 'type', 'author_id', 'possibly_sensitive', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'words', 'score_compound', 'sentiment', 'words_str'])
res = trainer.predict(raw_test_final)

In [ ]:
trainer.evaluate()

In [ ]:
!pip install ray[tune]

In [ ]:
from ray import tune

tune_config = {
        "num_train_epochs": tune.choice([4]),
        "batch_size": tune.choice([4,8,16,32])
    }

best_trial = trainer.hyperparameter_search(
    hp_space=lambda _: tune_config,
    direction="minimize",
    backend="ray",
    n_trials=10, # number of trials
)

In [ ]:
print(best_trial)

# Load Tweets

The dataframe of tweets contain the following columns:

- `id`: The unique identifier of the tweet
- `text`: The content of the tweet
- `type`: The type of tweet, which can be 'tweet', 'quoted', 'retweeted' or 'quoted__replied_to'
- `author_id`: The unique identifier of the author of the tweet
- `possibly_sensitive`: A boolean value indicating whether the tweet contains sensitive content
- `retweet_count`: The number of times the tweet has been retweeted
- `quote_count`: The number of times the tweet has been quoted
- `reply_count`: The number of times the tweet has been replied to
- `like_count`: The number of times the tweet has been liked
- `followers_count`: The number of followers of the author of the tweet
- `following_count`: The number of accounts the author of the tweet is following
- `tweet_count`: The total number of tweets made by the author of the tweet
- `listed_count`: The number of lists the author of the tweet is a member of
- `score_compound`:  A numerical value ranging from -1 to 1 indicating the overall sentiment of the tweet, where -1 represents  negative sentiment and 1 represents positive sentiment. **This is the target variable for the regression task.**
- `sentiment`: A categorical variable indicating the sentiment of the tweet, which can be 'negative', 'neutral' or 'positive'. **This is the target variable for the classification task.**




In [ ]:
import os
import pandas as pd
pd.set_option('display.max_rows', 100)

from wordcloud import WordCloud

import matplotlib.pyplot as plt
import seaborn as sns
from tueplots import bundles

plt.rcParams.update(bundles.icml2022())
import tueplots.constants.color.palettes as tue_palettes

from sentence_transformers import SentenceTransformer

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#CHANGE THIS PATH FOR YOU
%cd '/content/drive/MyDrive/Colab Notebooks/ML Twitter Project/Project Tutorials'
# %cd '/content/drive/MyDrive/coreML23/ML_2023/students'


In [ ]:
team_id = '36' #put your team id here
split = 'test_1' # replace by 'test_2' for FINAL submission

df = pd.read_csv('tweets_train.csv') # .sample(100) # we are subsampling 100 training data
df_test = pd.read_csv(f'tweets_{split}.csv') # .sample(200) # we are subsampling 200 test data


# Pre-process tweets

The following are the preprocessing steps we followed to get the `words` column from the original tweet, which corresponds to the `text` column of the dataframe.

- Remove punctuations, special characters, mentions, links, and numbers from the tweets.
- Convert all the tweets to lowercase.
- Tokenize the tweets into individual words.
- Remove stop words, such as "and", "the", "a", etc.
- Perform stemming or lemmatization on the remaining words to convert them to their base form.
- Filter out any words that occur infrequently in the corpus to reduce the dimensionality of the data.
- Create a bag of words representation of the tweets, where each tweet is represented as a vector of word frequencies.


**Note:** Lemmatization is a process in natural language processing where words are reduced to their base form, or lemma. This is done by removing inflections, such as pluralization or verb conjugation, and converting the word to its dictionary form. The result of this process is a word that is more easily recognizable, and can be used to improve the accuracy of NLP models, such as the LDA model. By lemmatizing the words in a corpus of text, the dimensionality of the data is reduced, and the relationships between words become clearer, making it easier to identify patterns and themes within the text.


In [ ]:
# joining list of words to create sentences

df['words_str'] = df['words'].apply(lambda words: ' '.join(eval(words)))
df_test['words_str'] = df_test['words'].apply(lambda words: ' '.join(eval(words)))


Join all of the preprocessed tweets together and create a world cloud of them to see most frequently used words among all tweets.

# Sentiment Analysis

**In this part, we will visualize the distribution of these possible sentiments in our dataset.**

Each tweet in our dataset have one of three sentiments (`sentiment`):

*   Positive
*   Neutral
*   Negative

Also, each tweet has a continous score (`score_compound`) between [-1,1] where -1 corresponds to negative and 1 corresponds to a positive sentinement.

In [ ]:
df_pos = df[df.sentiment == 'positive']
df_neu = df[df.sentiment == 'neutral']
df_neg = df[df.sentiment == 'negative']


num_total = len(df)
num_pos = len(df_pos)
num_neu = len(df_neu)
num_neg = len(df_neg)

print(f"Num. positive tweets: {num_pos} ({num_pos/num_total*100:.2f}%)")
print(f"Num. negative tweets: {num_neg} ({num_neg/num_total*100:.2f}%)")
print(f"Num. neutral tweets: {num_neu} ({num_neu/num_total*100:.2f}%)")


# Base Implementation

In [ ]:
%rm -r cardiffnlp

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import sklearn.metrics as skm
import numpy as np

In [ ]:
df['sentiment'] = df['sentiment'].map({'negative':0,'neutral':1,'positive':2})

In [ ]:
df['sentiment'].value_counts()

In [ ]:
for i in range(0,35):
    print(df['sentiment'][i])
    print(df['score_compound'][i])


In [ ]:
# Doing random over sampling for imbalance purposes

# from imblearn.over_sampling import RandomOverSampler

# ros = RandomOverSampler()
# train_x, train_y = ros.fit_resample(np.array(df['words_str']).reshape(-1,1), np.array(df['sentiment']).reshape(-1,1))

# train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['words_str', 'sentiment']);

# train_os['sentiment'].value_counts()

In [ ]:
#split X and y for training and validation purposes

# oversampled versions
# X = train_os['words_str'].values
# y = train_os['sentiment'].values

X = df['words_str'].values
y = df['sentiment'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

X_test = df_test['words_str'].values

In [ ]:
from sklearn import preprocessing
#one hot encoding

#save a copy
y_train_le = y_train.copy()
y_valid_le = y_val.copy()

ohe = preprocessing.OneHotEncoder()
y_train = ohe.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_valid = ohe.fit_transform(np.array(y_val).reshape(-1, 1)).toarray()

In [ ]:
print(f"TRAINING DATA: {X_train.shape[0]}\nVALIDATION DATA: {X_val.shape[0]}\nTESTING DATA: {X_test.shape[0]}" )

In [ ]:
from transformers import BertTokenizerFast
from transformers import TFBertModel

bert_model = TFBertModel.from_pretrained('bert-base-uncased')

In [ ]:
import tensorflow as tf
from tensorflow import keras
MAX_LEN = 128

def create_model(bert_model, max_len=MAX_LEN):

    ##params###
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, weight_decay=1e-7)
    loss = tf.keras.losses.CategoricalCrossentropy()
    accuracy = tf.keras.metrics.CategoricalAccuracy()

    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')

    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')

    embeddings = bert_model([input_ids,attention_masks])[1]

    output = tf.keras.layers.Dense(3, activation="softmax")(embeddings)

    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)

    model.compile(opt, loss=loss, metrics=accuracy)

    return model

In [ ]:
# tokenize
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

train_input_ids, train_attention_masks = tokenize(X_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize(X_val, MAX_LEN)
test_input_ids, test_attention_masks = tokenize(X_test, MAX_LEN)

In [ ]:
model = create_model(bert_model, MAX_LEN)
model.summary()

In [ ]:
# training

history_bert = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=4, batch_size=32)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

result_bert = model.predict([val_input_ids, val_attention_masks])

y_pred_bert =  np.zeros_like(result_bert)
y_pred_bert[np.arange(len(y_pred_bert)), result_bert.argmax(1)] = 1

print(y_pred_bert)

In [ ]:
y_val

In [ ]:
reg_res = list()

def proc(x, y):
  if y == 0:
    return x * -1
  if y == 1:
    return x - 1
  if y == 2:
    return x

for i in range(len(result_bert)):

  reg_res.append(
      proc(result_bert[i][y_val[i]], y_val[i])
  )

In [ ]:
print(reg_res)

In [ ]:
folder = '/content/drive/MyDrive/ML Twitter Project/Project Tutorials'
np.save(os.path.join(folder, f'{team_id}__{split}__reg_pred.npy'), np.array(reg_res))

In [ ]:
y_pred = np.argmax(y_pred_bert, axis=1)
print(y_pred.shape)

In [ ]:
print('\tClassification Report for BERT:\n\n',classification_report(y_valid,y_pred_bert, target_names=['negative', 'neutral', 'positive']))

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


At this point, we can use our model to predict sentiments scores of tweets from `X_test`, i.e. test set. Do not forget to encode them as well.

And save your predictions `y_hat` by naming it with the following format.

`<TEAM_ID>__<SPLIT>_reg_pred.npy`

Make sure that

`<TEAM_ID>` is your team id as given in CMS.

`<SPLIT>` is "test_1" during the semester and "test_2" for final submission. You will be notified when we need to move to "test_2".

In [ ]:
# Run this to save a file with your predictions on the test set to be submitted

# sentences = list(df_test.words_str.values)
# X_test = model.encode(sentences)

result_bert = model.predict([test_input_ids, test_attention_masks])

y_pred_bert =  np.zeros_like(result_bert)
y_pred_bert[np.arange(len(y_pred_bert)), result_bert.argmax(1)] = 1
y_pred = np.argmax(y_pred_bert, axis=1)

y_hat = y_pred

# Save the results with the format <TEAM_ID>__<SPLIT>_reg_pred.npy

folder = '/content/drive/MyDrive/ML Twitter Project/Project Tutorials'
np.save(os.path.join(folder, f'{team_id}__{split}__clf_pred.npy'), np.array(res))


In [ ]:
len(y_hat)

In [ ]:
res = []
for i in y_hat:
  if i == 0:
    res.append('negative')
  if i == 1:
    res.append('neutral')
  if i == 2:
    res.append('positive')

In [ ]:
np.array(res)

In [ ]:
result_bert.max(1)

In [ ]:
y_pred

In [ ]:
df['pred_numbers'] = y_hat
df['preds'] = df['pred_numbers'].map({'negative':0,'neutral':1,'positive':2})
df['regression_pred'] = (y_pred_bert * 2 - 1)

In [ ]:
# Run this to save a file with your predictions on the test set to be submitted

# sentences = list(df_test.words_str.values)
# X_test = model.encode(sentences)

result_bert = model.predict([test_input_ids, test_attention_masks])

y_pred = np.max(result_bert, axis=1)

y_hat = y_pred

# Save the results with the format <TEAM_ID>__<SPLIT>_reg_pred.npy

folder = '/content/drive/MyDrive/ML Twitter Project/Project Tutorials'
np.save(os.path.join(folder, f'{team_id}__{split}__reg_pred.npy'), y_hat)


In [ ]:
print(min(y_hat))

# Linear classification

In this part, we will solve a linear classification task to predict our target `sentiment`, i.e. sentiment class of tweets, using our features which are encodings of the tweets.



Reminders about macro and micro averaging:


In the context of computing F1-score, "macro" and "micro" averaging are two commonly used techniques to aggregate the per-class F1-scores.

**Micro-average**: Compute the F1-score globally by counting the total true positives, false negatives, and false positives over all classes, and then calculating precision, recall, and F1-score using these aggregated values.

**Macro-average**: Calculate the F1-score for each class separately, and then take the average of these per-class F1-scores.

The main difference between these two techniques is the way they treat class imbalance. Micro-average treats all classes equally, regardless of their size, while macro-average treats each class equally, regardless of the number of samples in that class.

Micro-average is often used when we care about overall performance across all classes, and we want to give more weight to the performance on larger classes. In contrast, macro-average is often used when we want to evaluate the performance on each class separately and give equal weight to each class.


In addition to micro and macro averaging, there is another common technique for computing the F1-score called **weighted averaging**.

**Weighted averaging** is similar to macro averaging in that it computes the per-class F1-score and then takes the average of these scores. However, unlike macro averaging, weighted averaging takes into account the number of samples in each class when computing the average. Specifically, the weighted average is computed as follows:

- Compute the F1-score for each class separately.
- Compute the weight for each class as the number of samples in that class divided by the total number of samples.
- Compute the weighted average of the per-class F1-scores, where each per-class F1-score is weighted by the weight of that class.

The weighted average is commonly used when the dataset is imbalanced, meaning that some classes have many more samples than others. In such cases, using the simple average (macro-average) would give too much weight to the smaller classes, while using micro-average would give too much weight to the larger classes. The weighted average strikes a balance between these two approaches by giving more weight to the classes with more samples while still taking into account the performance of all classes.


Now evaluate your model

In [ ]:
for split_name, dataset in zip(['train', 'validation'], df):
    X_i, y_i = dataset
    y_pred = clf.predict(X_i)
    print(f'\nSplit: {split_name}')
    print(skm.classification_report(y_i, y_pred))

At this point, we can use our model to predict sentiments scores of tweets from `X_test`, i.e. test set. Do not forget to encode them as well.

And save your predictions `y_hat` by naming it with the following format.

`<TEAM_ID>__<SPLIT>_clf_pred.npy`

Make sure that

`<TEAM_ID>` is your team id as given in CMS.

`<SPLIT>` is "test_1" during the semester and "test_2" for final submission. You will be notified when we need to move to "test_2".

In [ ]:
# Run this to save a file with your predictions on the test set to be submitted
sentences = list(df_test.words_str.values)
X_test = model.encode(sentences)
y_hat = clf.predict(X_test)

# Save the results with the format <TEAM_ID>__<SPLIT>_clf_pred.npy

folder = '/content/drive/MyDrive/Colab Notebooks/ML Twitter Project/Project Tutorials'
np.save(os.path.join(folder, f'{team_id}__{split}__clf_pred.npy'), y_hat)

In [ ]:
sentences

# Submission to CMS

Put your .npy files for both regression and classification tasks in the same zip file. Please name the file as `<TEAM_ID>.zip` and upload it to CMS system. It is essential that the files inside the .zip are named as follow:

`<TEAM_ID>__<SPLIT>__reg_pred.npy` \
`<TEAM_ID>__<SPLIT>__clf_pred.npy` \

Above, `<SPLIT>` should correspond to `test_1` for the leaderboard and `test_2` for the final submission.